In [3]:
# Cell 1 - setup
!pip install kaggle --quiet
import os, sys, time
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN = PROJECT_ROOT / "data" / "clean"
DATA_INTERIM = PROJECT_ROOT / "data" / "interim"

for p in [DATA_RAW, DATA_INTERIM, DATA_CLEAN]:
    p.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Raw data folder:", DATA_RAW)



Project root: /Users/aarush/projects/soil-health-ml
Raw data folder: /Users/aarush/projects/soil-health-ml/data/raw


In [4]:
# Cell 2 - place kaggle.json
from shutil import copyfile
kaggle_src = PROJECT_ROOT / "kaggle.json"   # adjust if you uploaded somewhere else

if kaggle_src.exists():
    os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
    copyfile(str(kaggle_src), os.path.expanduser("~/.kaggle/kaggle.json"))
    os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)
    print("Copied kaggle.json to ~/.kaggle/")
else:
    print("kaggle.json not found in project root. Upload it (download from your Kaggle account).")


Copied kaggle.json to ~/.kaggle/


In [5]:
# Cell 3 - download kaggle dataset (crop recommender example)
# Dataset: manikantasanjayv/crop-recommender-dataset-with-soil-nutrients
dataset_ref = "manikantasanjayv/crop-recommender-dataset-with-soil-nutrients"
!kaggle datasets download {dataset_ref} -p {str(DATA_RAW)} --unzip

# adjust filename if needed (list files)
list(DATA_RAW.iterdir())


Dataset URL: https://www.kaggle.com/datasets/manikantasanjayv/crop-recommender-dataset-with-soil-nutrients
License(s): CC0-1.0
  0%|                                               | 0.00/15.0k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 15.0k/15.0k [00:00<00:00, 22.9MB/s]


[PosixPath('/Users/aarush/projects/soil-health-ml/data/raw/dataset.csv')]

In [6]:
# Cell 4 - load crop_recommender.csv (adjust name if different)
kaggle_files = list(DATA_RAW.glob("*.csv"))
print("CSV files in data/raw:", kaggle_files)

# choose first or specify correct name
kaggle_csv = kaggle_files[0]
df_kaggle = pd.read_csv(kaggle_csv)
print("Kaggle dataset shape:", df_kaggle.shape)
df_kaggle.head()


CSV files in data/raw: [PosixPath('/Users/aarush/projects/soil-health-ml/data/raw/dataset.csv')]
Kaggle dataset shape: (620, 12)


,N,P,K,ph,EC,S,Cu,Fe,Mn,Zn,B,label
0,143,69,217,5.9,0.58,0.23,10.20,116.35,59.96,54.85,21.29,pomegranate
1,170,36,216,5.9,0.15,0.28,15.69,114.20,56.87,31.28,28.62,pomegranate
2,158,66,219,6.8,0.34,0.20,15.29,65.87,51.81,57.12,27.59,pomegranate
3,133,45,207,6.4,0.94,0.21,8.48,103.10,43.81,68.50,47.29,pomegranate
4,132,48,218,6.7,0.54,0.19,5.59,63.40,56.40,46.71,31.04,pomegranate


In [7]:
# Cell 5 - load SHC CSV(s)
# Put your SHC CSV into data/raw (e.g., SHC_samples.csv). If you have many, this will concat them.
shc_files = list(DATA_RAW.glob("*SHC*.csv")) + list(DATA_RAW.glob("*shc*.csv")) + list(DATA_RAW.glob("shc_*.csv"))
print("Found SHC-like files:", shc_files)

if not shc_files:
    print("No SHC CSV detected in data/raw. Upload your SHC CSV(s) into the data/raw folder.")
else:
    dfs = []
    for f in shc_files:
        try:
            dfs.append(pd.read_csv(f))
        except Exception as e:
            print("Error reading", f, e)
    df_shc = pd.concat(dfs, ignore_index=True)
    print("SHC combined shape:", df_shc.shape)
    df_shc.head()


Found SHC-like files: []
No SHC CSV detected in data/raw. Upload your SHC CSV(s) into the data/raw folder.


In [8]:
# Cell 6 - helper to standardize columns
def standardize_cols(df):
    # Lowercase, strip, replace spaces with underscore
    df = df.rename(columns=lambda c: str(c).strip())
    rename_map = {}
    # common alternate names -> target name
    mapping_candidates = {
        'nitrogen':'N', 'nitrogen (%)':'N', 'n':'N',
        'phosphorus':'P', 'phosphorus (%)':'P', 'p':'P',
        'potassium':'K', 'potassium (%)':'K', 'k':'K',
        'ph':'pH', 'pH':'pH',
        'ec':'EC', 'ec (dS/m)':'EC', 'ec(dS/m)':'EC',
        'organic carbon':'OC', 'organic_carbon':'OC', 'oc':'OC', 'organic_carbon(g/kg)':'OC'
    }
    cols = list(df.columns)
    for c in cols:
        lc = c.lower().strip()
        if lc in mapping_candidates:
            rename_map[c] = mapping_candidates[lc]
    df = df.rename(columns=rename_map)
    return df

df_kaggle = standardize_cols(df_kaggle)
if 'df_shc' in globals():
    df_shc = standardize_cols(df_shc)

print("Kaggle columns (after):", df_kaggle.columns.tolist())
if 'df_shc' in globals():
    print("SHC columns (after):", df_shc.columns.tolist())


Kaggle columns (after): ['N', 'P', 'K', 'pH', 'EC', 'S', 'Cu', 'Fe', 'Mn', 'Zn', 'B', 'label']


In [9]:
# Cell 7 - extract required columns and drop rows missing essential values
required = ['N','P','K','pH','EC','OC']

def select_and_clean(df, required):
    # keep only columns present from required
    present = [c for c in required if c in df.columns]
    df2 = df[present].copy()
    # Option: coerce non-numeric to NaN then drop rows missing too many values
    df2 = df2.apply(pd.to_numeric, errors='coerce')
    # drop rows where all required are NaN; you can be stricter later
    df2 = df2.dropna(subset=[c for c in ['N','P','K'] if c in df2.columns], how='all')
    return df2

df_k = select_and_clean(df_kaggle, required)
if 'df_shc' in globals():
    df_s = select_and_clean(df_shc, required)
else:
    df_s = pd.DataFrame(columns=df_k.columns)  # empty

print("Kaggle cleaned shape:", df_k.shape)
print("SHC cleaned shape:", df_s.shape)


Kaggle cleaned shape: (620, 5)
SHC cleaned shape: (0, 5)


In [10]:
# Cell 8 - merge and save
df_merged = pd.concat([df_k, df_s], ignore_index=True, sort=False)
df_merged.to_csv(DATA_CLEAN / "soil_data_clean.csv", index=False)
print("Merged shape:", df_merged.shape)
df_merged.head()


Merged shape: (620, 5)


/var/folders/mn/c4n_vz_92_s8kpf1y69hj4fw0000gn/T/ipykernel_54749/2801755622.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged = pd.concat([df_k, df_s], ignore_index=True, sort=False)


,N,P,K,pH,EC
0,143,69,217,5.9,0.58
1,170,36,216,5.9,0.15
2,158,66,219,6.8,0.34
3,133,45,207,6.4,0.94
4,132,48,218,6.7,0.54


In [11]:
# Cell 9 - quick stats and sample
display(df_merged.describe(include='all'))
# Save a smaller sample for quick iteration (optional)
df_merged.sample(min(2000, len(df_merged)), random_state=42).to_csv(DATA_INTERIM / "soil_sample_small.csv", index=False)
print("Saved sample to:", DATA_INTERIM / "soil_sample_small.csv")


,N,P,K,pH,EC
count,620.0,620.0,620.0,620.000000,620.000000
unique,135.0,90.0,238.0,NaN,NaN
top,129.0,76.0,61.0,NaN,NaN
freq,15.0,17.0,11.0,NaN,NaN
mean,NaN,NaN,NaN,6.106290,0.885468
std,NaN,NaN,NaN,0.693782,0.637214
min,NaN,NaN,NaN,4.500000,0.010000
25%,NaN,NaN,NaN,5.700000,0.297500
50%,NaN,NaN,NaN,6.100000,0.780000
75%,NaN,NaN,NaN,6.500000,1.390000


Saved sample to: /Users/aarush/projects/soil-health-ml/data/interim/soil_sample_small.csv
